In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
pst = pyemu.pst

In [ ]:
# specify a template directory (i.e. the PstFrom working folder)
template_ws = os.path.join("temp")
start_datetime="1-1-2008"
# instantiate PstFrom
pf = pyemu.utils.PstFrom(original_d=workspace, # where the model is stored
                            new_d=template_ws, # the PEST template folder
                            remove_existing=True, # ensures a clean start
                            longnames=False, # set False if using PEST/PEST_HP
                            # spatial_reference=sr, #the spatial reference we generated earlier
                            zero_based=False, # does the MODEL use zero based indices? For example, MODFLOW does NOT
                            start_datetime=start_datetime, # required when specifying temporal correlation between parameters
                            echo=False) # to stop PstFrom from writting lots of infromation to the notebook; experiment by setting it as True to see the difference; usefull for troubleshooting

In [ ]:
pd.read_csv(os.path.join(workspace,'output',"SV_HOBS_all._os"), sep = '\s+').columns

In [ ]:
from pyemu.pst.pst_utils import (
    SFMT,
    IFMT,
    FFMT,
    pst_config,
    parse_tpl_file,
    try_process_output_file,
)

In [ ]:

def modflow_hob_to_instruction_file(hob_file, ins_file=None):
    """write an instruction file for a modflow head observation file

    Args:
        hob_file (`str`): the path and name of the existing modflow hob file
        ins_file (`str`, optional): the name of the instruction file to write.
            If `None`, `hob_file` +".ins" is used.  Default is `None`.

    Returns:
        **pandas.DataFrame**: a dataFrame with control file observation information

    """

    hob_df = pd.read_csv(
        hob_file,
        sep=r"\s+",
        skiprows=1,
        header=None,
        names=["simval", "obsval", "obsnme", "date",'decimal_year'],
    )

    hob_df.loc[:, "obsnme"] = hob_df.obsnme.apply(str.lower)
    hob_df.loc[:, "ins_line"] = hob_df.obsnme.apply(lambda x: "l1 !{0:s}!".format(x))
    hob_df.loc[0, "ins_line"] = hob_df.loc[0, "ins_line"].replace("l1", "l2")

    if ins_file is None:
        ins_file = hob_file + ".ins"
    f_ins = open(ins_file, "w")
    f_ins.write("pif ~\n")
    f_ins.write(
        hob_df.loc[:, ["ins_line"]].to_string(
            col_space=0,
            columns=["ins_line"],
            header=False,
            index=False,
            formatters=[SFMT],
        )
        + "\n"
    )
    hob_df.loc[:, "weight"] = 1.0
    hob_df.loc[:, "obgnme"] = "obgnme"
    f_ins.close()
    return hob_df

In [ ]:
hob_df =modflow_hob_to_instruction_file(os.path.join(workspace,'output',"SV_HOBS_all._os"))

In [ ]:
template_ws = os.path.join("freyberg6_template")

hds_df = pf.add_observations(os.path.join('output',"SV_HOBS_all._os"), # the model output file to read
                             # new_d=template_ws, # the PEST template folder,
                            insfile="SV_hyd.hyd.ins4j", #optional, the instruction file name
                            index_cols=r"OBSERVATIONNAME", #column header to use as index; can also use column number (zero-based) instead of the header name
                            use_cols=r"SIMULATEDEQUIVALENT", #names of columns that include observation values; can also use column number (zero-based) instead of the header name
                             ofile_sep = '\s+',
                            prefix="hds") #prefix to all observation names; choose something logical and easy to find. We use it later on to select obsevrations

In [ ]:
import conda_scripts

In [ ]:
conda_scripts.arich_functions.get_flopy_model_spatial_reference()

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)
ml.external_path= 'rech_arrays'

In [ ]:
import flopy

In [ ]:
bas = flopy.modflow.ModflowRch.load(os.path.join(ml.model_ws,"SVIGFM_GSP.rch"),ml, )

In [ ]:
for i in range(590):
    bas.rech[i].how = 'openclose'

In [ ]:
bas.write_file()